# Day 5

In [1]:
import os

In [79]:
input_file_path = os.path.join(".", "day05.txt")
with open(input_file_path, 'r') as reader:
    input_data = reader.read()

## Part 1 + Part 2
Part 1: Count the number of points where two or more lines overlap, horizontal and vertical lines only
Part 2: Including diagonal lines

### The naive way
Using an int array

In [80]:
import numpy as np

instructions = []
width = height = 0

# First pass: parse all input lines and determine field dimensions
for instruction in input_data.split("\n"):
    if len(instruction) < 1:
        continue

    origin, target = instruction.split(" -> ")
    coordinates = list(map(int, origin.split(",") + target.split(",")))
    instructions.append(coordinates)

    width = max(width, coordinates[0], coordinates[2])
    height = max(height, coordinates[1], coordinates[3])

width += 1
height += 1

field = np.zeros((width, height), dtype="uint8")
field2 = field.copy()
print(f"Field dimension is {width:,} x {height:,}, using {field.nbytes / 1024:0,.1f}KB of memory")


Field dimension is 990 x 991, using 958.1KB of memory


In [81]:
# Second pass: mark lines
for (x1, y1, x2, y2) in instructions:

    # Vertical line
    if x1 == x2:
        if y1 > y2:
            y1, y2 = y2, y1

        field[y1:y2+1, x1] += 1
        field2[y1:y2+1, x1] += 1

    # Horizontal line
    elif y1 == y2:
        if x1 > x2:
            x1, x2 = x2, x1

        field[y1, x1:x2+1] += 1
        field2[y1, x1:x2+1] += 1

    # Diagonal line
    else:
        # Assume lines are 45 degrees
        x_delta = x2 - x1
        y_delta = y2 - y1

        if abs(x_delta) != abs(y_delta):
            raise RuntimeError(f"{x1}, {y1} -> {x2}, {y2} is not of the form y = x")

        if x_delta > 0:
            x_deltas = range(x_delta + 1)
        else:
            x_deltas = range(0, x_delta - 1, -1)

        if y_delta > 0:
            y_deltas = range(y_delta + 1)
        else:
            y_deltas = range(0, y_delta - 1, -1)

        for x, y in zip(x_deltas, y_deltas):
            field2[y1 + y, x1 + x] += 1

print(f"Part 1: more than two overlaps: {np.sum(field > 1):,}")
print(f"Part 2: more than two overlaps: {np.sum(field2 > 1):,}")


Part 1: more than two overlaps: 5,084
Part 2: more than two overlaps: 17,882


### The counting way
Assuming the field is sparse relative to lines occupying the field,
keep count of occupied spots only

In [82]:
from collections import defaultdict

field = defaultdict(lambda: 0)
field2 = defaultdict(lambda: 0)

for instruction in input_data.split("\n"):
    if len(instruction) < 1:
        continue

    origin, target = instruction.split(" -> ")
    x1, y1, x2, y2 = list(map(int, origin.split(",") + target.split(",")))

    # Vertical line
    if x1 == x2:
        if y1 > y2:
            y1, y2 = y2, y1

        for y in range(y1, y2 + 1):
            field[(x1, y)] += 1
            field2[(x1, y)] += 1

    # Horizontal line
    elif y1 == y2:
        if x1 > x2:
            x1, x2 = x2, x1

        for x in range(x1, x2 + 1):
            field[(x, y1)] += 1
            field2[(x, y1)] += 1

    # Diagonal line
    else:
        if abs(x2 - x1) != abs(y2 - y1):
            raise RuntimeError(f"{x1}, {y1} -> {x2}, {y2} is not of the form y = x")

        x_incr = 1 if x2 > x1 else -1
        y_incr = 1 if y2 > y1 else -1

        for x, y in zip(range(x1, x2 + x_incr, x_incr), range(y1, y2 + y_incr, y_incr)):
            field2[(x, y)] += 1

overlaps = [coordinate for coordinate, count in field.items() if count > 1]
overlaps2 = [coordinate for coordinate, count in field2.items() if count > 1]
print(f"Part 1: more than two overlaps: {len(overlaps):,}")
print(f"Part 2: more than two overlaps: {len(overlaps2):,}")

Part 1: more than two overlaps: 5,084
Part 2: more than two overlaps: 17,882
